# ***Play with langChain Build in tools***

### *Search Tool*

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_community.tools import DuckDuckGoSearchRun, TavilySearchResults
search = TavilySearchResults(k=1)  
response = search.invoke("Who is the current government of Bangladesh?")
print(response)

[{'url': 'https://en.wikipedia.org/wiki/Government_of_Bangladesh', 'content': "Headquarters | Bangladesh Secretariat,Dhaka\nWebsite | bangladesh.gov.bd\nThe executive government is led by the prime minister, who selects all the remaining ministers. The prime minister and the other most senior ministers belong to the supreme decision-making committee, known as the Cabinet. After the resignation of Sheikh Hasina in August 2024, the current interim government is led by Dr. Muhammad Yunus as chief adviser. [...] The government of the People's Republic of Bangladesh (Bengali: গণপ্রজাতন্ত্রী বাংলাদেশ সরকার, romanized:\xa0Gôṇôprôjātôntrī Bāṅlādēś Sôrkār) is the central government of Bangladesh. The government was constituted by the Constitution of Bangladesh comprising the executive (the president, prime minister and cabinet), the legislature (the Jatiya Sangsad), and the judiciary (the Supreme Court).  Bangladesh is a unitary state[1] and the central government has the authority to govern ov

In [22]:
print(search.name)
print(search.description)
print(search.args)

tavily_search_results_json
A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


## Shell Tool

In [11]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()
result = shell_tool.invoke('whoami')
print(result)

Executing command:
 whoami
mdsloq\lenovo



c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


### *[LangChain Tools list Documentation](https://python.langchain.com/docs/integrations/tools/)*

# ***Build Custom Tools***

- step1: define a function with argument type
- step2: must add description what is the purpose of this function
- step3: add tool decorator and its make this function as runnable

In [13]:
from langchain_core.tools import tool

In [14]:
#step1
def multiply(a, b):
    return a*b

In [18]:
#step2 and 3

@tool
def multiply(a:int, b:int)->int:
    """multiply two integer number and return an integer number"""
    return a*b

In [17]:
multiply.invoke(
    {
        "a": 5,
        "b": 6
    }
)

30

In [20]:
print(multiply.name,"\n",multiply.description,"\n",multiply.args)

multiply 
 multiply two integer number and return an integer number 
 {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


## When llm get this tool its see this info about this tools

In [23]:
print(multiply.args_schema.model_json_schema())

{'description': 'multiply two integer number and return an integer number', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


### Mostly uses method to creating tools
- using `@tool` decorator
- using Structured tool and Pydantic
- Using BaseTool class

# **Make tool using Structured and Pydantic**

In [26]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field


class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number of add")
    
def multiply_func(a: int, b: int)->int:
    return a*b

In [27]:
## making the tools
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two integer number",
    args_schema=MultiplyInput
)

In [29]:
multiply_tool.invoke({
    'a': 5,
    'b': 5
})

25

In [30]:
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

multiply
Multiply two integer number
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number of add', 'required': True, 'title': 'B', 'type': 'integer'}}


In [31]:
print(multiply_tool.args_schema.model_json_schema())

{'properties': {'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number of add', 'required': True, 'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'MultiplyInput', 'type': 'object'}


## Using BaseToolClass

In [34]:
from langchain_core.tools import BaseTool
from typing import Type

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [35]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"
    
    args_schema: Type[BaseModel] = MultiplyInput
    
    def _run(self, a: int, b: int) -> int:
        return a*b

In [36]:
mul_tool = MultiplyTool()

In [38]:
mul_tool.invoke(
    {
        "a": 10,
        'b': 4
    }
)

40

In [39]:
print(mul_tool.args_schema.model_json_schema())

{'properties': {'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'MultiplyInput', 'type': 'object'}


# Toolkit
- a toolkit is just a collection of related tools and that serve a common purpose package together for convenience and reusability
- Example of toolkit is GoogleDriveToolKit
    - `GoogleDriveCreateFileTool`: Upload a file
    - `GoogleDriveSearchFileTool`: Search a file
    - `GoogleDriveReadFileTool`: Read the content of the file

## Build custom tool

In [40]:
@tool
def add(a: int, b: int)->int:
    "add two integer number"
    return a+b

@tool
def multiply(a: int, b:int) -> int:
    """multiply two integer number"""
    return a*b

In [41]:
class MathToolKit:
    def get_tools(self):
        return [add, multiply]

In [43]:
toolkit = MathToolKit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "==>", tool.description, "==>", tool.args)

add ==> add two integer number ==> {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
multiply ==> multiply two integer number ==> {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
